<a href="https://colab.research.google.com/github/werowe/HypatiaAcademy/blob/master/spark/spark_youtube_german_channels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
!wget https://raw.githubusercontent.com/Annepikson/HypatiaAcademy/refs/heads/master/numpy/top-1000-most-subscribed-youtube-channels-in-germany%204.csv youtube.csv

--2025-02-26 06:20:23--  https://raw.githubusercontent.com/Annepikson/HypatiaAcademy/refs/heads/master/numpy/top-1000-most-subscribed-youtube-channels-in-germany%204.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 70079 (68K) [text/plain]
Saving to: ‘top-1000-most-subscribed-youtube-channels-in-germany 4.csv.2’

top-1000-most-subsc 100%[===================>]  68.44K  --.-KB/s    in 0.01s   

2025-02-26 06:20:23 (4.50 MB/s) - ‘top-1000-most-subscribed-youtube-channels-in-germany 4.csv.2’ saved [70079/70079]

--2025-02-26 06:20:23--  http://youtube.csv/
Resolving youtube.csv (youtube.csv)... failed: Name or service not known.
wget: unable to resolve host address ‘youtube.csv’
FINISHED --2025-02-26 06:20:23--
Total wall clock time: 0.3s
Downloaded: 1 files,

In [44]:
!head "top-1000-most-subscribed-youtube-channels-in-germany 4.csv"

﻿rank,Youtuber,subscribers,video views,video count,category,started
1,Tsuriki Show,"34,100,000","42,490,526,838","4,739",Entertainment,2019
2,Kidibli (Kinder Spielzeug Kanal),"29,600,000","15,673,364,837","1,236",Entertainment,2015
3,Kurzgesagt – In a Nutshell,"23,600,000","3,145,706,013",271,Education,2013
4,boxtoxtv,"23,500,000","18,303,986,629","1,559",Comedy,2022
5,HaerteTest,"19,500,000","3,420,864,412","1,712",Science & Technology,2011
6,Noel Robinson,"18,600,000","10,752,582,783","1,639",Entertainment,2015
7,FAMILY BOOMS,"16,800,000","16,225,259,066","1,587",Entertainment,2021
8,Talking Angela,"13,300,000","4,116,834,514",345,Entertainment,2014
9,Ice Cream Rolls,"12,400,000","1,709,743,695","2,725",Entertainment,2011


In [45]:
from pyspark.sql.functions import col
from pyspark.sql import SparkSession

file="top-1000-most-subscribed-youtube-channels-in-germany 4.csv"


spark = SparkSession.builder.appName("youtube").getOrCreate()


df = spark.read.csv(file, sep=',', header=True,   inferSchema=True, encoding="UTF-8" )


df.printSchema()



root
 |-- rank: integer (nullable = true)
 |-- Youtuber: string (nullable = true)
 |-- subscribers: string (nullable = true)
 |-- video views: string (nullable = true)
 |-- video count: string (nullable = true)
 |-- category: string (nullable = true)
 |-- started: integer (nullable = true)



In [46]:
df.show()

+----+--------------------+-----------+--------------+-----------+--------------------+-------+
|rank|            Youtuber|subscribers|   video views|video count|            category|started|
+----+--------------------+-----------+--------------+-----------+--------------------+-------+
|   1|        Tsuriki Show| 34,100,000|42,490,526,838|      4,739|       Entertainment|   2019|
|   2|Kidibli (Kinder S...| 29,600,000|15,673,364,837|      1,236|       Entertainment|   2015|
|   3|Kurzgesagt – In a...| 23,600,000| 3,145,706,013|        271|           Education|   2013|
|   4|            boxtoxtv| 23,500,000|18,303,986,629|      1,559|              Comedy|   2022|
|   5|          HaerteTest| 19,500,000| 3,420,864,412|      1,712|Science & Technology|   2011|
|   6|       Noel Robinson| 18,600,000|10,752,582,783|      1,639|       Entertainment|   2015|
|   7|        FAMILY BOOMS| 16,800,000|16,225,259,066|      1,587|       Entertainment|   2021|
|   8|      Talking Angela| 13,300,000| 

In [47]:
from pyspark.sql.functions import regexp_replace, col

df = df.withColumn("subscribers", regexp_replace(col("subscribers"), ",", "").cast("long"))

df = df.withColumn("video views", regexp_replace(col("video views"), ",", "").cast("long"))

df.printSchema()

df.show()

root
 |-- rank: integer (nullable = true)
 |-- Youtuber: string (nullable = true)
 |-- subscribers: long (nullable = true)
 |-- video views: long (nullable = true)
 |-- video count: string (nullable = true)
 |-- category: string (nullable = true)
 |-- started: integer (nullable = true)

+----+--------------------+-----------+-----------+-----------+--------------------+-------+
|rank|            Youtuber|subscribers|video views|video count|            category|started|
+----+--------------------+-----------+-----------+-----------+--------------------+-------+
|   1|        Tsuriki Show|   34100000|42490526838|      4,739|       Entertainment|   2019|
|   2|Kidibli (Kinder S...|   29600000|15673364837|      1,236|       Entertainment|   2015|
|   3|Kurzgesagt – In a...|   23600000| 3145706013|        271|           Education|   2013|
|   4|            boxtoxtv|   23500000|18303986629|      1,559|              Comedy|   2022|
|   5|          HaerteTest|   19500000| 3420864412|      1,71

In [48]:
df = df.withColumnRenamed("video views", "video_views")
df = df.withColumnRenamed("video count", "video_count")


In [49]:
df.createOrReplaceTempView("videos")

In [50]:

sql="describe videos"

spark.sql(sql).show()

+-----------+---------+-------+
|   col_name|data_type|comment|
+-----------+---------+-------+
|       rank|      int|   NULL|
|   Youtuber|   string|   NULL|
|subscribers|   bigint|   NULL|
|video_views|   bigint|   NULL|
|video_count|   string|   NULL|
|   category|   string|   NULL|
|    started|      int|   NULL|
+-----------+---------+-------+



In [51]:
sql="""
    SELECT * from videos
"""

spark.sql(sql).show()




+----+--------------------+-----------+-----------+-----------+--------------------+-------+
|rank|            Youtuber|subscribers|video_views|video_count|            category|started|
+----+--------------------+-----------+-----------+-----------+--------------------+-------+
|   1|        Tsuriki Show|   34100000|42490526838|      4,739|       Entertainment|   2019|
|   2|Kidibli (Kinder S...|   29600000|15673364837|      1,236|       Entertainment|   2015|
|   3|Kurzgesagt – In a...|   23600000| 3145706013|        271|           Education|   2013|
|   4|            boxtoxtv|   23500000|18303986629|      1,559|              Comedy|   2022|
|   5|          HaerteTest|   19500000| 3420864412|      1,712|Science & Technology|   2011|
|   6|       Noel Robinson|   18600000|10752582783|      1,639|       Entertainment|   2015|
|   7|        FAMILY BOOMS|   16800000|16225259066|      1,587|       Entertainment|   2021|
|   8|      Talking Angela|   13300000| 4116834514|        345|       

In [55]:
topChannels= spark.sql("""
    SELECT Youtuber, sum(subscribers) as sum_subscribers, sum(video_views) as sum_video_views
    from videos
    group by  Youtuber
    order by sum_video_views desc
""")
topChannels.show()



+--------------------+---------------+---------------+
|            Youtuber|sum_subscribers|sum_video_views|
+--------------------+---------------+---------------+
|        Tsuriki Show|       34100000|    42490526838|
|            boxtoxtv|       23500000|    18303986629|
|        FAMILY BOOMS|       16800000|    16225259066|
|Kidibli (Kinder S...|       29600000|    15673364837|
|       German Spidey|       10900000|    11280067476|
|       Noel Robinson|       18600000|    10752582783|
|      ag_soccer team|        9050000|     9196775078|
|         ArkivaShqip|        4610000|     7678417494|
|  Rammstein Official|        8600000|     6634639951|
|             Paluten|        5530000|     5564401210|
|           Uyen Ninh|        2920000|     5363263070|
|      ProsafiaGaming|        4700000|     4919466523|
|         TikTokTunes|        6990000|     4713201758|
|        SKITS German|        5150000|     4640958734|
|    Nik Wild Animals|        5490000|     4475179345|
|        R

In [59]:
topChannels.coalesce(1).write.csv("videos_partition", header=True)

In [60]:
!ls videos_partition

part-00000-427162e2-f140-45a8-ac65-871b8fc0f2a0-c000.csv  _SUCCESS


In [62]:
!head videos_partition/part-00000-427162e2-f140-45a8-ac65-871b8fc0f2a0-c000.csv

Youtuber,sum_subscribers,sum_video_views
Tsuriki Show,34100000,42490526838
boxtoxtv,23500000,18303986629
FAMILY BOOMS,16800000,16225259066
Kidibli (Kinder Spielzeug Kanal),29600000,15673364837
German Spidey,10900000,11280067476
Noel Robinson,18600000,10752582783
ag_soccer team,9050000,9196775078
ArkivaShqip,4610000,7678417494
Rammstein Official,8600000,6634639951
